In [1]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName("tree").getOrCreate()

In [7]:
data = spark.read.csv('College.csv',inferSchema=True, header=True)

In [8]:
data.describe().show()

+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+
|summary|              School|Private|              Apps|            Accept|          Enroll|         Top10perc|         Top25perc|      F_Undergrad|      P_Undergrad|          Outstate|        Room_Board|             Books|          Personal|               PhD|          Terminal|         S_F_Ratio|       perc_alumni|          Expend|         Grad_Rate|
+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------

In [9]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [2]:
data.head(1)

NameError: name 'data' is not defined

In [11]:
from pyspark.ml.feature import VectorAssembler

In [13]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [17]:
assembler = VectorAssembler(inputCols=['Apps',
                                      'Accept', 
                                       'Enroll', 
                                       'Top10perc',
                                      'Top25perc',
                                      'F_Undergrad',
                                      'P_Undergrad',
                                      'Outstate',
                                      'Room_Board',
                                      'Books',
                                      'Personal',
                                      'PhD',
                                      'Terminal',
                                      'S_F_Ratio',
                                      'perc_alumni',
                                      'Expend',
                                      'Grad_Rate'],outputCol='features')

In [18]:
output = assembler.transform(data)

In [19]:
from pyspark.ml.feature import StringIndexer

In [20]:
indexer = StringIndexer(inputCol='Private',outputCol='PrivateIndex')

In [21]:
output_fixed = indexer.fit(output).transform(output)

In [23]:
final_data = output_fixed.select('features', 'PrivateIndex')

In [24]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [26]:
from pyspark.ml.classification import (DecisionTreeClassifier, GBTClassifier, RandomForestClassifier)

In [27]:
from pyspark.ml import Pipeline

In [28]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')

In [29]:
rfc = RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features')

In [30]:
gbt = GBTClassifier(labelCol='PrivateIndex',featuresCol='features')

In [31]:
dtc_model = dtc.fit(train_data)

In [32]:
rfc_model = rfc.fit(train_data)

In [33]:
gbt_model = gbt.fit(train_data)

In [34]:
dtc_preds = dtc_model.transform(test_data)

In [36]:
rfc_preds = rfc_model.transform(test_data)

In [38]:
gbt_preds = gbt_model.transform(test_data)

In [39]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [40]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [41]:
print('DTC')
print(my_binary_eval.evaluate(dtc_preds))

DTC
0.8055061113088882


In [42]:
print('RFC')
print(my_binary_eval.evaluate(rfc_preds))

RFC
0.9673667971994777


In [43]:
gbt_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [44]:
rfc_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [52]:
my_binary_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndex',rawPredictionCol='prediction')

In [46]:
print("GBT")
print(my_binary_eval2.evaluate(gbt_preds))

GBT
0.916933665598671


In [47]:
rfc_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [48]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [49]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex',metricName='accuracy')

In [50]:
rfc_acc = acc_eval.evaluate(rfc_preds)

In [51]:
rfc_acc

0.9339622641509434